In [1]:
from bs4 import BeautifulSoup     #网页解析，获取数据\n",
import re       #正则表达式，进行文字匹配\n",
import urllib.request,urllib.error      #制定URL，获取网页数据\n",
# import xlwt     #进行excel操作\n",
import sqlite3  #进行SQLite数据库操作\n",
import pandas as pd #存储数据及处理数据\n",
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import csv
from math import *
import pymysql
# from geopy.distance import geodesic
# from pyproj import Proj
pd.set_option("display.max_columns", 100)
import itertools

from scipy import optimize
import matplotlib.pyplot as plt

In [10]:
# nansha guangzhou
df_base=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\1990_2020_2022.csv',dtype={'地区编码':'str','年份':'str','area_city':'str'})

In [42]:
df_base.loc[(df_base['地区编码']=='440115')&(df_base['行业代码'].str.startswith('C36'))]['年份'].value_counts()

年份
2022    5
2019    4
2020    4
2018    3
2016    2
2017    2
2015    2
2013    1
2014    1
2012    1
2011    1
Name: count, dtype: int64

In [45]:
df_base_2022=df_base.loc[df_base['年份']=='2022']
df_base_2022_=df_base_2022.loc[df_base_2022['行业代码'].isin(ls_ind4)]

# df_gz=df_base_2022_.loc[df_base_2022_['area_city']==4401]
# df_gz_ns=df_base_2022_.loc[df_base_2022_['地区编码']==440115]

In [46]:
df_base_2022_.loc[(df_base_2022_['地区编码']=='440115')&(df_base_2022_['行业代码']=='C3670')]

,年份,地区编码,行业代码,企业数量,area_city
16785621,2022,440115,C3670,59,4401


In [34]:
df_base_2022_['企业数量'].sum()

4757509

In [48]:
df_2.loc[(df_2['地区编码']=='440115')&(df_2['行业代码']=='C3670')]

,地区编码,行业代码,num,num_dis,num_total_ind,total_num,rca
1092208,440115,C3670,59,124977,104782,38783772,0.174737


In [23]:
# df_gz=df_base_2022_.loc[df_base_2022_['area_city']=='4401']
df_gz_dis=df_base_2022_.groupby(['地区编码','行业代码'])['企业数量'].sum().reset_index()
df_gz_dis_num=df_base_2022_.groupby(['地区编码'])['企业数量'].sum().reset_index()
df_gz_dis_num.rename(columns={'企业数量':'num_dis'},inplace=True)
df_gz_dis.rename(columns={'企业数量':'num'},inplace=True)
df_base_2022_ind=df_base_2022_.groupby(['行业代码'])['企业数量'].sum().reset_index()
df_base_2022_ind.rename(columns={'企业数量':'num_total_ind'},inplace=True)
total_num=df_base_2022_ind['num_total_ind'].sum()
df_1=pd.merge(left=df_gz_dis,right=df_gz_dis_num,on=['地区编码'],how='left')
print(len(df_1),len(df_gz_dis))
df_2=pd.merge(left=df_1,right=df_base_2022_ind,on=['行业代码'],how='left')
print(len(df_2),len(df_1))
df_2.loc[:,'total_num']=total_num
df_2.loc[:,'rca']=(df_2['num']/df_2['num_dis'])/(df_2['num_total_ind']/df_2['total_num'])
df_2.to_csv(r'E:\project_data\22nansha\tupu\gz_all_dis_rca.csv',index=False,mode='w')

1653071 1653071
1653071 1653071


In [28]:
df_dis_code=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',dtype='str',sheet_name='t_admin_division_code_dis')

df_3=pd.merge(left=df_2,right=df_dis_code,on=['地区编码'],how='left')
df_4=pd.merge(left=df_3,right=df_small,left_on=['行业代码'],right_on=['Id'],how='left')
df_4.to_csv(r'E:\project_data\22nansha\tupu\all_dis_rca.csv',index=False,mode='w')

In [32]:
df_4.loc[df_4['short_name1']=='广州市'].to_csv(r'E:\project_data\22nansha\tupu\gz_all_dis_rca.csv',index=False,mode='w')

In [29]:
df_4.loc[df_4['']]

,地区编码,行业代码,num,num_dis,num_total_ind,total_num,rca,type_4,admin_name,short_name,short_name1,Id,Label,Id.1
0,110000,G5449,1,1,39450,38783772,983.112091,1100,北京市,北京,北京市,G5449,其他道路运输辅助活动,G5449
1,110100,A0111,1,23521,200489,38783772,0.008224,1101,北京市,北京,北京市,A0111,稻谷种植,A0111
2,110100,A0112,1,23521,145363,38783772,0.011343,1101,北京市,北京,北京市,A0112,小麦种植,A0112
3,110100,A0113,1,23521,73128,38783772,0.022548,1101,北京市,北京,北京市,A0113,玉米种植,A0113
4,110100,A0119,1,23521,124603,38783772,0.013233,1101,北京市,北京,北京市,A0119,其他谷物种植,A0119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653066,659904,F5291,1,22,190106,38783772,9.273241,6599,一牧场,一牧场,昆玉市,F5291,流动货摊零售,F5291
1653067,659904,F5299,1,22,1022795,38783772,1.723609,6599,一牧场,一牧场,昆玉市,F5299,其他未列明零售业,F5299
1653068,659904,J6820,1,22,26698,38783772,66.031116,6599,一牧场,一牧场,昆玉市,J6820,财产保险,J6820
1653069,659904,L7291,1,22,121331,38783772,14.529665,6599,一牧场,一牧场,昆玉市,L7291,旅行社及相关服务,L7291


In [23]:
df_base_2022=df_base.loc[df_base['年份']==2022]
df_base_2022_=df_base_2022.loc[df_base_2022['行业代码'].isin(ls_ind4)]

df_gz=df_base_2022_.loc[df_base_2022_['area_city']==4401]
df_gz_ns=df_base_2022_.loc[df_base_2022_['地区编码']==440115]
df_gz_ns

ls_df=[df_base_2022_,df_gz]
ls_name=['total','gz']
num_city=df_gz_ns['企业数量'].sum()
for i in range(len(ls_df)):
    df_i=ls_df[i]
    df_2=df_i.groupby(['行业代码'])['企业数量'].sum().reset_index()
    df_2=df_2.rename(columns={'企业数量':'num_ind'})
    num_total=df_i['企业数量'].sum()

    df_result1=pd.merge(left=df_gz_ns,right=df_2,on=['行业代码'],how='left')
    print('合并城市数据前后',len(df_result1),len(df_gz_ns))
    df_result1.loc[:,'num_city']=num_city
    df_result1.loc[:,'num_total']=num_total

    df_result1.loc[:,'rca']=(df_result1['企业数量']/df_result1['num_city'])/(df_result1['num_ind']/df_result1['num_total'])
    df_result1.to_csv('E:\\project_basedata\\2023090601启信宝企业统计数据\\fz_ns'+ls_name[i]+'.csv',index=False)

合并城市数据前后 791 791
合并城市数据前后 791 791


In [ ]:
# 根据城市编码进行筛选，且把四个直辖市对应的编码统一成xx00，北京11开头，天津12开头，上海31开头，重庆50开头
ls_municipality=['11','12','31','50']
for i in ls_municipality:
    df_0_.loc[df_0_['area_city'].str.startswith(i),'area_city']=i+'00'
df_0_1=df_0_.loc[df_0_['area_city'].isin(ls_city_code)]
print(len(df_0_1),len(df_0_))

# 根据小类行业进行筛选
df_0_ind4=df_0_1.loc[df_0_1['行业代码'].isin(ls_ind4)]
df_0_ind4=df_0_ind4.rename(columns={'行业代码':'ind'})
df_0_ind4=pd.merge(left=df_0_ind4,right=df_city_code,left_on=['area_city'],right_on=['type_4'],how='left')
df_0_ind4=df_0_ind4.drop(columns=['type_4'])
print('小类数据量',len(df_0_ind4),len(df_0_1))

ls_df=[df_0_ind4]
df_ind_nodes=[df_small]
ls_df_name=['small']

In [3]:
# 南沙区 440115
df_city_code.loc[df_city_code['admin_name']]

,type_4,admin_name
0,1100,北京市
1,1200,天津市
2,1301,石家庄市
27,1302,唐山市
46,1303,秦皇岛市
...,...,...
4060,8102,九龙
4066,8103,新界
4076,8201,澳门半岛
4082,8202,氹仔岛


In [4]:
# 城市的地区编码读取
df_city_code=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',dtype='str',sheet_name='t_admin_division_code_city')
df_city_code=df_city_code.drop_duplicates(['type_4'],keep='first')
ls_city_code=df_city_code['type_4'].unique().tolist()
# 中类和小类行业读取
df_ind=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='industry_new')
df_ind.loc[:,'ind3']=df_ind['中类代码'].str[:4]
ls_ind4=df_ind['小类类别代码'].unique().tolist()
ls_ind3=df_ind['ind3'].unique().tolist()
df_big=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='big')
df_medium=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='medium')
df_small=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='small')

In [ ]:
# 计算逻辑2：
start=time.time()
ls_year=['1990','1995','2000','2005','2010','2015','2020','2022']
# 计算各年各城市各行业rca
for i in range(len(ls_df)):
    df_i=ls_df[i]
    print(ls_df_name[i])
    
    df_1=df_i.groupby(['年份','area_city'])['企业数量'].sum().reset_index()
    df_1=df_1.rename(columns={'企业数量':'num_city'})
    df_2=df_i.groupby(['年份','ind'])['企业数量'].sum().reset_index()
    df_2=df_2.rename(columns={'企业数量':'num_ind'})
    df_3=df_i.groupby(['年份'])['企业数量'].sum().reset_index()
    df_3=df_3.rename(columns={'企业数量':'num_year'})
    df_4=df_i.groupby(['年份','area_city','ind'])['企业数量'].sum().reset_index()
    df_4=df_4.rename(columns={'企业数量':'num'})

    df_result1=pd.merge(left=df_4,right=df_1,on=['年份','area_city'],how='outer')
    print('合并城市数据前后',len(df_result1),len(df_4))
    df_result2=pd.merge(left=df_result1,right=df_2,on=['年份','ind'],how='outer')
    print('合并行业数据前后',len(df_result2),len(df_4))
    df_result3=pd.merge(left=df_result2,right=df_3,on=['年份'],how='outer')
    print('合并年份数据前后',len(df_result3),len(df_4))

    df_result3.loc[:,'rca']=(df_result3['num']/df_result3['num_city'])/(df_result3['num_ind']/df_result3['num_year'])
    df_result3.to_csv('E:\\project_basedata\\2023090801产业生长图谱\\'+ls_df_name[i]+'_rca_dup_1990_2022.csv',index=False,mode='w')

# 筛选各城市的优势行业，输出【年份，城市，优势行业，rca】，选取rca阈值为1.6，数据量约为17.2w/49.8w
    df_result4=df_result3.loc[df_result3['rca']>=1.6,['年份','area_city','ind','rca']]
    df_result5=df_result4.groupby(['年份','area_city'])['ind'].apply(lambda x:';'.join(x)).reset_index()
    ls_result=[]
    n=1
    dq_ind=deque(df_result4['ind'].unique().tolist())
    while n<=len(dq_ind):
        print(n)
        ind_source=dq_ind.popleft()
        for a in dq_ind:
            ind_target=a
            for b in range(len(ls_year)-1):
                year_b=ls_year[b]
                year_b_=ls_year[b+1]
                stage_b=year_b+'_'+year_b_
                df_b=df_result5.loc[(df_result5['年份']==year_b)&(df_result5['ind'].str.contains(ind_source))&(~df_result5['ind'].str.contains(ind_target))]
                ls_n_total=df_b['area_city'].unique().tolist()
                
                if len(ls_n_total)!=0:
                    df_result_b=df_result5.loc[(df_result5['年份']==year_b_)&(df_result5['ind'].str.contains(ind_target))&(df_result5['area_city'].isin(ls_n_total))]
                    if len(df_result_b)!=0:
                        weight_b=len(df_result_b)/len(ls_n_total)
                        ls_b=[ind_source,ind_target,weight_b,len(df_result_b),len(ls_n_total),stage_b]
                        ls_result.append(ls_b)
                    
        dq_ind.append(ind_source)
        n+=1
    df_result=pd.DataFrame(ls_result,columns=['Source','Target','Weight','best_num','stage_num','stage'])
    df_result.to_csv('E:\\project_basedata\\2023090801产业生长图谱\\'+ls_df_name[i]+'_graph_1990_2022.csv',index=False,mode='w')
    
    ls_ind_i=df_result4['ind'].unique().tolist()
    df_nodes=pd.DataFrame(ls_ind_i,columns=['Id'])
    df_nodes_stan=df_ind_nodes[i]
    df_nodes_result=pd.merge(left=df_nodes,right=df_nodes_stan,on=['Id'],how='left')
    df_nodes_result.to_csv('E:\\project_basedata\\2023090801产业生长图谱\\'+ls_df_name[i]+'_nodes_1990_2022.csv',index=False,mode='w')
    end=time.time()
    print(end-start)

In [ ]:
def func_poly_1(x,a,b):
    return a*x+b

def r_square(ydata,ydata_fit):
    s_list=[(ydata_fit[x]-ydata[x])**2 for x in range(len(ydata))]
    sse=sum(s_list)
    y_mean=sum(ydata)/len(ydata)
    sst=sum([(y-y_mean)**2 for y in ydata])
    return 1-(sse/sst)

In [ ]:
# 计算逻辑4：
# 计算逻辑4：
# 1根据每个城市每个行业的rca历年变化散点数据，拟合出一条最合适的曲线
# 曲线求拐点，进行曲线分段
# 2每个城市的各行业两两算相关系数，pearson[皮尔森]、spearman[斯皮尔曼] 和 kendall[肯德尔]
# 3找到负相关的行业数据绘图查验一下

start=time.time()
for b in range(len(ls_df)):
    df_i=ls_df[b]
    print(ls_df_name[b])
    
    df_1=df_i.groupby(['年份','area_city'])['企业数量'].sum().reset_index()
    df_1=df_1.rename(columns={'企业数量':'num_city'})
    df_2=df_i.groupby(['年份','ind'])['企业数量'].sum().reset_index()
    df_2=df_2.rename(columns={'企业数量':'num_ind'})
    df_3=df_i.groupby(['年份'])['企业数量'].sum().reset_index()
    df_3=df_3.rename(columns={'企业数量':'num_year'})
    df_4=df_i.groupby(['年份','area_city','ind'])['企业数量'].sum().reset_index()
    df_4=df_4.rename(columns={'企业数量':'num'})

    df_result1=pd.merge(left=df_4,right=df_1,on=['年份','area_city'],how='outer')
    print('合并城市数据前后',len(df_result1),len(df_4))
    df_result2=pd.merge(left=df_result1,right=df_2,on=['年份','ind'],how='outer')
    print('合并行业数据前后',len(df_result2),len(df_4))
    df_result3=pd.merge(left=df_result2,right=df_3,on=['年份'],how='outer')
    print('合并年份数据前后',len(df_result3),len(df_4))

    df_result3.loc[:,'rca']=(df_result3['num']/df_result3['num_city'])/(df_result3['num_ind']/df_result3['num_year'])
    df_result3=pd.merge(left=df_result3,right=df_city_code,left_on=['area_city'],right_on=['type_4'],how='left')
    df_result3=df_result3.drop(columns=['type_4'])
    df_result3=pd.merge(left=df_result3,right=df_small,left_on=['ind'],right_on=['Id'],how='left')
    df_result3=df_result3.drop(columns=['Id'])
    df_result3.to_csv('E:\\project_basedata\\2023090801产业生长图谱\\'+ls_df_name[b]+'_rca_1990_2020.csv',index=False,mode='w')
    end=time.time()
    print(end-start)
    
    ls_de=['N','O','P','Q','R','S','T']
    df_result3.loc[:,'ind_cap']=df_result3['ind'].str[0]
    df_result4=df_result3.loc[~(df_result3['ind_cap'].isin(ls_de))]

    ls_city=df_result4['area_city'].unique().tolist()
    df_result=pd.DataFrame()
    count=0
    for i in ls_city:
        df_a=df_result4.loc[(df_result4['area_city']==i)]
        ls_ind=df_a['ind'].unique().tolist()
        for a in ls_ind:
            df_ia=df_a.loc[(df_a['ind']==a)]
            df_1=df_ia.loc[:,['年份','rca']].set_index(['年份']).transpose()
            df_1.loc[:,'area_city']=i
            df_1.loc[:,'ind']=a
            df_result=pd.concat([df_result,df_1],axis=0)
        count+=1
        print(count,'/',len(ls_city))
    end=time.time()
    print(end-start)
    
    df_result=pd.merge(left=df_result,right=df_city_code,left_on=['area_city'],right_on=['type_4'],how='left')
    df_result=df_result.drop(columns=['type_4'])
    df_result=pd.merge(left=df_result,right=df_small,left_on=['ind'],right_on=['Id'],how='left')
    df_result=df_result.drop(columns=['Id'])
    df_result.reset_index(drop=True).to_csv('E:\\project_basedata\\2023090801产业生长图谱\\'+ls_df_name[b]+'_rca_revolve_1990_2020.csv',
                                            index=False,mode='w')

In [ ]:
# 计算逻辑4

# 频数统计，统计全国各城市出现负相关（-10<a<-0.10）的行业ind_o和ind_d对的出现次数除以总城市数，
# 条件：
# 1）斜率在-10<a<-0.10
# 2）r方大于0.8，pearson的p值在0.05以下，pearson指数
# 3）点数据量在6个及以上

# 1027计算每个城市每个行业的rca变化趋势，用线性拟合的方式，确定该行业rca是增是降，rca增的为D，rca降的为O，需要rca数据量大于3才算数
# 1027统计每组负相关联系的两个行业同时出现在的城市数量

# 注：产业关系不要重复，ab和ba不要重复出现a

df_result=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\small_rca_revolve_1990_2020.csv',dtype={'area_city':'str'})
ls_c1=df_result.columns.tolist()
ls_c2=[x for x in ls_c1 if x.startswith('2') or x.startswith('1')]
for i in range(len(df_result)):
    df_i=df_result.loc[i,ls_c2]
    df_i_=df_i.dropna()
    if len(df_i_)>4:
        xdata=[int(x) for x in df_i_.index.tolist()]
        ydata=df_i_.values.tolist()
        try:
            popt1,pcov=optimize.curve_fit(func_poly_1,xdata,ydata,maxfev=20000)
            if True in np.isnan(popt1):
                print('empty:',i,df_result.loc[i,'admin_name'],df_result.loc[i,'Label'])
                continue
        except:
            print('error:',i,df_result.loc[i,'admin_name'],df_result.loc[i,'Label'])
            continue
        df_result.loc[i,'line_coef_1']=popt1[0]
        df_result.loc[i,'line_coef_2']=popt1[1]
        df_result.loc[i,'r_square']=r_square(ydata,func_poly_1(np.array(xdata),popt1[0],popt1[1]))
        
df_result.to_csv(r'E:\project_basedata\2023090801产业生长图谱\small_rca_revolve_1990_2020_poly.csv',index=False,mode='w')

In [ ]:
# &(df_i['line_coef_1']>=-10)&(df_i['line_coef_1']<=-0.1)
ls_base=os.listdir(r'E:\project_basedata\2023090801产业生长图谱\pearson')
df_result_10=pd.DataFrame()
count=1
for i in ls_base:
    # print(i)
    df_i=pd.read_csv('E:\\project_basedata\\2023090801产业生长图谱\\pearson\\'+i,dtype={'area_city':'str'})
    df_i_fil=df_i.loc[(df_i['p']<0.05)&(df_i['r2']>0.8)&(~((df_i['ind_o'].str.startswith('F'))&(df_i['ind_d'].str.startswith('F'))))].reset_index(drop=True)
    if len(df_i_fil)>0:
        # print('ok')
        ls=df_i_fil.loc[:,['ind_o','ind_d']].values.tolist()
        df_ind_od=pd.DataFrame([sorted(x) for x in ls],columns=['ind_o_','ind_d_'])
        df_i_fil.loc[:,['ind_o_','ind_d_']]=df_ind_od
        df_result_10=pd.concat([df_result_10,df_i_fil],axis=0)
        count+=1
    else:
        print(i)
    # if len(df_i_fil.loc[(df_i_fil['ind_o'].str.startswith('F'))&(df_i_fil['ind_d'].str.startswith('F'))])>0:
        # print(1)
print(count)

In [ ]:
df_result_10_=df_result_10.reset_index(drop=True)
df_result_10_.loc[df_result_10_['ind_o']!=df_result_10_['ind_o_'],'line_coef_1_']=1/df_result_10_['line_coef_1']
df_result_10_.loc[df_result_10_['line_coef_1_'].isnull(),'line_coef_1_']=df_result_10_['line_coef_1']

df_result_count=df_result_10_.groupby(['ind_o_','ind_d_'])['admin_name'].count().reset_index()
df_result_count.loc[:,'rate']=df_result_count['admin_name']/364
df_result_city=df_result_10_.groupby(['ind_o_','ind_d_'])['admin_name'].apply(lambda x: ';'.join(x)).reset_index()
df_result_city=df_result_city.rename(columns={'admin_name':'city_name'})
df_total_result=pd.merge(left=df_result_count,right=df_result_city,on=['ind_o_','ind_d_'],how='inner')
df_result_cof1=df_result_10_.groupby(['ind_o_','ind_d_'])['line_coef_1_'].mean().reset_index()
df_total_result_1=pd.merge(left=df_total_result,right=df_result_cof1,on=['ind_o_','ind_d_'],how='inner')
print(len(df_result_count),len(df_result_city),len(df_total_result),len(df_total_result_1))

df_result_result2=pd.merge(left=df_total_result_1,right=df_small,left_on=['ind_o_'],right_on=['Id'],how='left')
df_result_result2=df_result_result2.drop(columns=['Id'])
df_result_result2=df_result_result2.rename(columns={'Label':'ind_name_o'})
df_result_result3=pd.merge(left=df_result_result2,right=df_small,left_on=['ind_d_'],right_on=['Id'],how='left')
df_result_result3=df_result_result3.drop(columns=['Id'])
df_result_result3=df_result_result3.rename(columns={'Label':'ind_name_d'})

df_result_result3.to_csv(r'E:\\project_basedata\\2023090801产业生长图谱\\负相关联系频数.csv',index=False,mode='w')

In [ ]:
df_result_result3=pd.read_csv('E:\\project_basedata\\2023090801产业生长图谱\\负相关联系频数.csv')

df_result_10_=df_result_10.reset_index(drop=True)
df_result_10_1=df_result_10_.groupby(['ind_o_','ind_d_'])['area_city'].count().reset_index()
df_result_10_1_=df_result_10_.groupby(['ind_o_','ind_d_'])['admin_name'].apply(lambda x: ';'.join(x)).reset_index()
df_result_10_1=pd.merge(left=df_result_10_1,right=df_result_10_1_,on=['ind_o_','ind_d_'],how='inner')
df_result_10_1=df_result_10_1.rename(columns={'admin_name':'area_city_name'})
df_result_10_2=pd.merge(left=df_result_result3,right=df_result_10_1,on=['ind_o_','ind_d_'],how='left')
# df_result_10_2=df_result_10_2.drop(columns=['ind_o','ind_d'])

start = time.time()
for i in range(len(df_result_10_2)):
    city_i=df_result_10_2.loc[i,'city_name']
    ls_city_i=city_i.split(';')
    ind_o_=df_result_10_2.loc[i,'ind_o_']
    ind_d_=df_result_10_2.loc[i,'ind_d_']
    ls_slope_o=[]
    ls_slope_d=[]
    for a in ls_city_i:
        df_slope_o=df_result.loc[(df_result['admin_name']==a)&(df_result['ind']==ind_o_)&(df_result['r_square']>0.8)]['line_coef_1'].values.tolist()
        if len(df_slope_o)>0:
            ls_slope_o.append(str(round(df_slope_o[0],4)))
        else:
            ls_slope_o.append('no')
        df_slope_d=df_result.loc[(df_result['admin_name']==a)&(df_result['ind']==ind_d_)&(df_result['r_square']>0.8)]['line_coef_1'].values.tolist()
        if len(df_slope_d)>0:
            ls_slope_d.append(str(round(df_slope_d[0],4)))
        else:
            ls_slope_d.append('no')
    df_result_10_2.loc[i,'slope_o']=';'.join(ls_slope_o)
    df_result_10_2.loc[i,'slope_d']=';'.join(ls_slope_d)
    ls_slp_o_num=[float(x) for x in ls_slope_o if x!='no']
    ls_slp_d_num=[float(x) for x in ls_slope_d if x!='no']
    if len(ls_slp_o_num)>0:
        if max(ls_slp_o_num)<0:
            df_result_10_2.loc[i,'real_o']=df_result_10_2.loc[i,'ind_name_o']
            df_result_10_2.loc[i,'real_d']=df_result_10_2.loc[i,'ind_name_d']
        elif min(ls_slp_o_num)>0:
            df_result_10_2.loc[i,'real_o']=df_result_10_2.loc[i,'ind_name_d']
            df_result_10_2.loc[i,'real_d']=df_result_10_2.loc[i,'ind_name_o']
            
    if len(ls_slp_d_num)>0:
        if max(ls_slp_d_num)<0:
            df_result_10_2.loc[i,'real_o']=df_result_10_2.loc[i,'ind_name_d']
            df_result_10_2.loc[i,'real_d']=df_result_10_2.loc[i,'ind_name_o']
        elif min(ls_slp_d_num)>0:
            df_result_10_2.loc[i,'real_o']=df_result_10_2.loc[i,'ind_name_o']
            df_result_10_2.loc[i,'real_d']=df_result_10_2.loc[i,'ind_name_d']
        
    print(i,'/',len(df_result_10_2))
end = time.time()
runTime = end - start
print("运行时间：", runTime)

In [ ]:
df_result_10_2.loc[df_result_10_2['ind_name_o']==df_result_10_2['real_o'],'real_o_code']=df_result_10_2['ind_o_']
df_result_10_2.loc[df_result_10_2['ind_name_o']==df_result_10_2['real_o'],'real_d_code']=df_result_10_2['ind_d_']
df_result_10_2.loc[(df_result_10_2['ind_name_o']!=df_result_10_2['real_o'])&(df_result_10_2['real_o'].notnull()),'real_o_code']=df_result_10_2['ind_d_']
df_result_10_2.loc[(df_result_10_2['ind_name_o']!=df_result_10_2['real_o'])&(df_result_10_2['real_o'].notnull()),'real_d_code']=df_result_10_2['ind_o_']

df_result_10_2.to_csv(r'E:\project_basedata\2023090801产业生长图谱\负相关联系方向编码及频数.csv',index=False,mode='w')

In [ ]:
ls_de=['D','H','N','O','P','Q','R','S','T','F']
df_result_10_2.loc[:,'ind_o_type']=df_result_10_2['ind_o_'].str[0]
df_result_10_2.loc[:,'ind_d_type']=df_result_10_2['ind_d_'].str[0]
df_new_result=df_result_10_2.loc[~((df_result_10_2['ind_d_type'].isin(ls_de)))]
df_new_result_1=df_new_result.loc[~((df_new_result['ind_name_o'].str.contains('其他'))|(df_new_result['ind_name_d'].str.contains('其他')))]
df_new_result_2=df_new_result_1.copy()
df_new_result_2.loc[:,'weight']=(df_new_result_1['admin_name']*df_new_result_1['admin_name'])/df_new_result_1['area_city']
df_new_result_2=df_new_result_2.reset_index(drop=True)
df_new_result_2.to_csv(r'E:\project_basedata\2023090801产业生长图谱\负相关联系方向编码及平方频数筛选基底.csv',index=False,mode='w')

In [ ]:
# 项目用

In [204]:
# 新思路：起点弄一个按步延展的dataframe，终点弄一个，两个取相同点构成链路，差别是起点——共同点和共同点——终点
# 预测模拟产品的由起点向终点的生长路径
def find_potential_nodes_grow(ls_adv, ls_forbidden, wight_value, df_product_net):
    # ls_adv是本地优势产品，list
    # ls_local是本地所有产品列表，list
    # forbidden是预先设置的排除产业，list
    # wight_value是设定的阈值，float
    # df_product_net是产品上下游联系网络，dataframe

    _district_industryCOs = ls_adv
    ls_forbidden = ls_forbidden
    _simulate_list = [_district_industryCOs]
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        # 把所有优势产品输入_exist_list
        for k in range(0, len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []

        for j in range(len(_simulate_list[-1])):
            _industryCO = _simulate_list[-1][j]
            if _industryCO not in ls_forbidden:
                # 判断该优势产品属于本地产品，且不属于本地排除产品，然后从该优势产品开始向外找，一步联系值大于等于wight_value的节点
                _df_industryCO_proximity = df_product_net.loc[(df_product_net['primary_code'] == _industryCO)&(df_product_net['primary_weight'] >= wight_value)]
                if len(_df_industryCO_proximity) > 0:
                    # 取出该一步联系所能联系到的所有产品节点
                    # _industryCOs_1 = _df_industryCO_proximity.loc[:, 'primary_code'].unique()
                    _industryCOs_2 = _df_industryCO_proximity.loc[:, 'related_code'].unique()

                    # for m in range(len(_industryCOs_1)):
                    #     _industryCO_ = _industryCOs_1[m]
                    #     # 如果该产品节点不是已经找过的产品（作为起点的产品），也不属于本地排除产品，则加入到_possible_list
                    #     if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                    #         _possible_list.append(_industryCO_)
                    #         ls_a = []
                    #         ls_a.append(t)
                    #         ls_a.append(_industryCO)
                    #         ls_a.append(_industryCO_)
                    #         ls_forcast.append(ls_a)
                    #         c = c + 1

                    for m in range(len(_industryCOs_2)):
                        _industryCO_ = _industryCOs_2[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _weight_=_df_industryCO_proximity.loc[_df_industryCO_proximity['related_code']==_industryCO_]['primary_weight'].tolist()[0]
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_a.append(_weight_)
                            ls_forcast.append(ls_a)
                            c = c + 1
        if c == 0:
            # 当c等于0时，说明一步联系找到的都是已有产品，无法再继续找了
            flag = 0
        else:
            # 当c不等于0时，说明还可以继续往下找
            flag = 1
            t = t + 1

            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

# 生长图谱，反向寻找，根联系
def find_potential_nodes_root(ls_adv, ls_forbidden, wight_value, df_product_net):
    # ls_adv是本地优势产品，list
    # ls_local是本地所有产品列表，list
    # forbidden是预先设置的排除产业，list
    # wight_value是设定的阈值，float
    # df_product_net是产品上下游联系网络，dataframe

    _district_industryCOs = ls_adv
    ls_forbidden = ls_forbidden
    _simulate_list = [_district_industryCOs]
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        # 把所有优势产品输入_exist_list
        for k in range(0, len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []

        for j in range(len(_simulate_list[-1])):
            _industryCO = _simulate_list[-1][j]
            if _industryCO not in ls_forbidden:
                # 判断该优势产品属于本地产品，且不属于本地排除产品，然后从该优势产品开始向外找，一步联系值大于等于wight_value的节点
                _df_industryCO_proximity = df_product_net.loc[(df_product_net['related_code'] == _industryCO)&(df_product_net['primary_weight'] >= wight_value)]
                if len(_df_industryCO_proximity) > 0:
                    # 取出该一步联系所能联系到的所有产品节点
                    _industryCOs_1 = _df_industryCO_proximity.loc[:, 'primary_code'].unique()
                    # _industryCOs_2 = _df_industryCO_proximity.loc[:, 'related_code'].unique()

                    for m in range(len(_industryCOs_1)):
                        _industryCO_ = _industryCOs_1[m]
                        # 如果该产品节点不是已经找过的产品（作为起点的产品），也不属于本地排除产品，则加入到_possible_list
                        if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                            _weight_=_df_industryCO_proximity.loc[_df_industryCO_proximity['primary_code']==_industryCO_]['primary_weight'].tolist()[0]
                            _possible_list.append(_industryCO_)
                            ls_a = []
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_a.append(_weight_)
                            ls_forcast.append(ls_a)
                            c = c + 1

                    # for m in range(len(_industryCOs_2)):
                    #     _industryCO_ = _industryCOs_2[m]
                    #     # 如果该产品节点不是已经找过的产品（作为起点的产品），则加入到_possible_list
                    #     if _industryCO_ not in _exist_list and _industryCO not in ls_forbidden:
                    #         _possible_list.append(_industryCO_)
                    #         ls_a = []
                    #         ls_a.append(t)
                    #         ls_a.append(_industryCO)
                    #         ls_a.append(_industryCO_)
                    #         ls_forcast.append(ls_a)
                    #         c = c + 1
        if c == 0:
            # 当c等于0时，说明一步联系找到的都是已有产品，无法再继续找了
            flag = 0
        else:
            # 当c不等于0时，说明还可以继续往下找
            flag = 1
            t = t + 1

            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast


# 根据find_potential_nodes_grow找到的dataframe梳理出一条生长链路
def path_1(start,end,df_grow_net):
    df_grow_net=df_grow_net.drop(columns=['t'])
    flag=1
    count=1
    ls_columns=['origin']
    ls_result_3=[start]
    df_2=pd.DataFrame(ls_result_3,columns=ls_columns)

    while flag==1:
        df_1=pd.DataFrame()
        for i in ls_result_3:
            df_start=df_grow_net.loc[df_grow_net['origin']==i]
            df_1=pd.concat([df_1,df_start],axis=0)
        if len(df_1)==0:
            flag=0
        else:
            df_2=pd.merge(left=df_2,right=df_1,on=['origin'],how='outer')
            ls_columns.insert(-1,'col'+str(count))
            df_2.columns=ls_columns
            ls_result_3=df_2['origin'].unique().tolist()
            count+=1
            if end in ls_result_3:
                flag=0
    return df_2

def path_2_grow(start,end,df_grow_net):
    df_grow_net=df_grow_net.drop(columns=['t'])
    flag=1
    count=1
    columns=['origin', 'destination','weight']
    ls_col=['origin_g1', 'destination_g1','weight_g1']
    df_i=df_grow_net.loc[df_grow_net['origin']==start]
    loc=np.where(df_i==end)
    if len(df_i)==0:
        return pd.DataFrame()
    elif len(loc[0])>0:
        df_i.columns=ls_col
        des=df_i.columns.tolist()[-2]
        return df_i.loc[df_i[des]==end]
    else:
        while flag==1:
            des=df_i.columns.tolist()[-2]
            ls_o=df_i[des].unique().tolist()
            df_a=df_grow_net.loc[df_grow_net['origin'].isin(ls_o)]
            if len(df_a)>0:
                count+=1
                df_r=pd.merge(left=df_i,right=df_a,left_on=[des],right_on=['origin'],how='right')
                ls_col.extend([x+'_g'+str(count) for x in columns])
                df_r.columns=ls_col
                df_i=df_r
                loc=np.where(df_i==end)
                if len(loc[0])>0:
                    flag=0
            else:
                flag=0
        des=df_i.columns.tolist()[-2]
        return df_i.loc[df_i[des]==end]
    
def path_2_root(start,end,df_grow_net):
    df_grow_net=df_grow_net.drop(columns=['t'])
    flag=1
    count=1
    columns=['origin', 'destination','weight']
    ls_col=['origin_r1', 'destination_r1','weight_r1']
    df_i=df_grow_net.loc[df_grow_net['destination']==start]
    loc=np.where(df_i==end)
    if len(df_i)==0:
        return pd.DataFrame()
    elif len(loc[0])>0:
        df_i.columns=ls_col
        des=df_i.columns.tolist()[-2]
        return df_i.loc[df_i[des]==end]
    else:
        while flag==1:
            des=df_i.columns.tolist()[-2]
            ls_o=df_i[des].unique().tolist()
            df_a=df_grow_net.loc[df_grow_net['destination'].isin(ls_o)]
            if len(df_a)>0:
                count+=1
                df_r=pd.merge(left=df_i,right=df_a,left_on=[des],right_on=['destination'],how='right')
                ls_col.extend([x+'_r'+str(count) for x in columns])
                df_r.columns=ls_col
                df_i=df_r
                loc=np.where(df_i==end)
                if len(loc[0])>0:
                    flag=0
            else:
                flag=0
        des=df_i.columns.tolist()[-2]
        return df_i.loc[df_i[des]==end]

In [87]:
df_new_result_2=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\负相关联系方向编码及平方频数筛选基底.csv')
df_new_result_2.rename(columns={'real_o_code':'primary_code','real_d_code':'related_code','weight':'primary_weight'},inplace=True)
ls_se=['A','B','C','E','G','I','J','K','L','M']

df_start=pd.read_excel(r'E:\project_basedata\2023090801产业生长图谱\ind_example\现状和未来产业.xlsx',sheet_name='现状')
df_end=pd.read_excel(r'E:\project_basedata\2023090801产业生长图谱\ind_example\现状和未来产业.xlsx',sheet_name='未来')
ls_adv_1=df_start['小类类别代码'].unique().tolist()
ls_adv_2=df_end['小类类别代码'].unique().tolist()

In [4]:
df_new_result_2_filt=df_new_result_2.loc[(df_new_result_2['ind_o_type'].isin(ls_se))&(df_new_result_2['ind_d_type'].isin(ls_se))]
df_new_result_2_filt=df_new_result_2_filt.loc[(df_new_result_2['primary_code'].notnull())&(df_new_result_2['related_code'].notnull())&
                                              (~(df_new_result_2['primary_code'].isin(ls_adv_1)&df_new_result_2['related_code'].isin(ls_adv_1)))&
                                              (~(df_new_result_2['primary_code'].isin(ls_adv_2)&df_new_result_2['related_code'].isin(ls_adv_2)))].reset_index(drop=True)

In [96]:
start='C2110'
end='E4831'
df_grow_net=df_result_2
flag=1
count=1
columns=['t', 'origin', 'destination','weight']
ls_col=['t_1', 'origin_1', 'destination_1','weight_1']
df_i=df_grow_net.loc[df_grow_net['destination']==start]
loc=np.where(df_i==end)
if len(df_i)==0:
    df_re1=pd.DataFrame()
elif len(loc[0])>0:
    df_i.columns=ls_col
    des=df_i.columns.tolist()[-2]
    df_re2=df_i.loc[df_i[des]==end]
else:
    while flag==1:
        des=df_i.columns.tolist()[-2]
        ls_o=df_i[des].unique().tolist()
        df_a=df_grow_net.loc[df_grow_net['destination'].isin(ls_o)]
        if len(df_a)>0:
            count+=1
            df_r=pd.merge(left=df_i,right=df_a,left_on=[des],right_on=['destination'],how='right')
            ls_col.extend([x+'_'+str(count) for x in columns])
            df_r.columns=ls_col
            df_i=df_r
            loc=np.where(df_i==end)
            if len(loc[0])>0:
                flag=0
        else:
            flag=0
    des=df_i.columns.tolist()[-2]
    df_re3=df_i.loc[df_i[des]==end]

In [205]:
start=a
end=i
df_grow_net=df_result_1
flag=1
count=1
columns=['t', 'origin', 'destination','weight']
ls_col=['t_1', 'origin_1', 'destination_1','weight_1']
df_i=df_grow_net.loc[df_grow_net['origin']==start]
loc=np.where(df_i==end)
if len(df_i)==0:
    df1=pd.DataFrame()
elif len(loc[0])>0:
    df_i.columns=ls_col
    des=df_i.columns.tolist()[-2]
    df2=df_i.loc[df_i[des]==end]
else:
    while flag==1:
        des=df_i.columns.tolist()[-2]
        ls_o=df_i[des].unique().tolist()
        df_a=df_grow_net.loc[df_grow_net['origin'].isin(ls_o)]
        if len(df_a)>0:
            count+=1
            df_r=pd.merge(left=df_i,right=df_a,left_on=[des],right_on=['origin'],how='right')
            ls_col.extend([x+'_'+str(count) for x in columns])
            df_r.columns=ls_col
            df_i=df_r
            loc=np.where(df_i==end)
            if len(loc[0])>0:
                flag=0
        else:
            flag=0
    des=df_i.columns.tolist()[-2]
    df3=df_i.loc[df_i[des]==end]

In [210]:
# 新思路：起点弄一个按步延展的dataframe，终点弄一个，两个取相同点构成链路，差别是起点——共同点和共同点——终点
# ls_adv_1=['C3611']
# ls_adv_2=['C1711']

ls_forbidden=[]
wight_value=0
# df_product_net=df_net.rename(columns={'Source':'primary_code','Target':'related_code','Weight':'primary_weight'})
ls_result_1=find_potential_nodes_grow(ls_adv_1, ls_forbidden, wight_value, df_new_result_2_filt)
df_result_1=pd.DataFrame(ls_result_1,columns=['t', 'origin', 'destination','weight'])
ls_result_2=find_potential_nodes_root(ls_adv_2, ls_forbidden, wight_value, df_new_result_2_filt)
df_result_2=pd.DataFrame(ls_result_2,columns=['t', 'destination', 'origin','weight'])

ls1=df_result_1['destination'].unique().tolist()
ls2=df_result_1['origin'].unique().tolist()
ls_1=ls1+ls2
ls_1=[x for x in ls_1 if x not in ls_adv_1]

ls3=df_result_2['destination'].unique().tolist()
ls4=df_result_2['origin'].unique().tolist()
ls_2=ls3+ls4
ls_2=[x for x in ls_2 if x not in ls_adv_2]

ls_mid=[x for x in ls_1 if x in ls_2]
print(len(ls_adv_1),len(ls_adv_2),len(ls_mid))

28 37 1093


In [216]:
ls_total_path_result=[]
df_total_path_result=pd.DataFrame()
for a in ls_adv_1:
    for i in ls_mid:
        df_first_half=path_2_grow(a,i,df_result_1)
        if len(df_first_half)>0:
            for b in ls_adv_2:
                df_sec_half=path_2_root(b,i,df_result_2)
                if len(df_sec_half)>0:
                    leftcol=df_first_half.columns.tolist()[-2]
                    rightcol=df_sec_half.columns.tolist()[-2]
                    df_path=pd.merge(left=df_first_half,right=df_sec_half,left_on=[leftcol],right_on=[rightcol],how='inner')
                    ls_path_re=df_path.values.tolist()
                    ls_total_path_result.extend(ls_path_re)
                    if len(ls_total_path_result)%1000==0:
                        print(len(ls_total_path_result))

1000
12000
15000
17000
20000
22000
27000
28000
29000
35000
37000
43000
44000
49000
52000
58000
67000
72000
73000
77000
81000
86000
90000
92000
94000
104000
107000
108000
109000
111000
115000
116000
117000
118000
124000
125000
130000
133000
134000
136000
137000
144000
145000


In [228]:
df_re=pd.DataFrame(ls_total_path_result)
df_re.fillna(0.0,inplace=True)
ls_val=df_re.values.tolist()
df_re.loc[:,'val']=[sum([a for a in x if type(a)==float]) for x in ls_val]

df_re_new=df_re.loc[df_re[9]!=0]
ls_val_new=df_re_new.values.tolist()

ls_nodes=list(set([y for x in ls_val_new for y in x if type(y)==str]))
df_graph_base=df_new_result_2_filt.loc[:,['primary_code','related_code','primary_weight']]
df_graph=pd.DataFrame()

for i in ls_val_new:
    i_=[x for x in i if type(x)==str]
    ls_pair=[i_[x:x+2] for x in range(0,len(i_),2)]
    for a in ls_pair:
        ind1=a[0]
        ind2=a[1]
        df_a=df_graph_base.loc[((df_graph_base['primary_code']==ind1)&(df_graph_base['related_code']==ind2))|((df_graph_base['primary_code']==ind2)&(df_graph_base['related_code']==ind1))]
        df_graph=pd.concat([df_graph,df_a],axis=0)
# df_graph=df_graph_base.loc[(df_graph_base['primary_code'].isin(ls_nodes))&(df_graph_base['related_code'].isin(ls_nodes))]

df_graph=df_graph.drop_duplicates()
ls_total_nodes1=df_graph['primary_code'].unique().tolist()
ls_total_nodes2=df_graph['related_code'].unique().tolist()
ls_total_nodes=list(set(ls_total_nodes1+ls_total_nodes2))

df_graph=df_graph.rename(columns={'primary_code':'Source','related_code':'Target','primary_weight':'Weight'})
df_nodes=pd.DataFrame(ls_total_nodes,columns=['Id'])
df_nodes.loc[df_nodes['Id'].isin(ls_adv_1),'type']='start'
df_nodes.loc[df_nodes['Id'].isin(ls_adv_2),'type']='end'
df_nodes.loc[df_nodes['Id'].isin(ls_mid),'type']='mid'
df_nodes.loc[:,'ind_type0']=df_nodes['Id'].str[0]
df_nodes_1=pd.merge(left=df_nodes,right=df_small,on=['Id'],how='left')
df_nodes_1.to_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\nodes1.csv',index=False)
df_graph.to_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\graph1.csv',index=False)

In [234]:
df_re_new.to_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\route_base_new.csv',index=False)

In [254]:
df_re1=df_re.drop_duplicates()
df_re_short=df_re1.loc[df_re1[9]==0]
# df_re_short.sort_values(by=['val'],ascending=False)
df0=pd.merge(left=df_re_short,right=df_small,left_on=[0],right_on=['Id'],how='left',suffixes=('_0','_1'))
df1=pd.merge(left=df0,right=df_small,left_on=[1],right_on=['Id'],how='left',suffixes=('_0','_1'))
df2=pd.merge(left=df1,right=df_small,left_on=[3],right_on=['Id'],how='left',suffixes=('_0','_1'))
df3=pd.merge(left=df2,right=df_small,left_on=[4],right_on=['Id'],how='left',suffixes=('_2','_3'))
df4=pd.merge(left=df3,right=df_small,left_on=[6],right_on=['Id'],how='left',suffixes=('_4','_5'))
df5=pd.merge(left=df4,right=df_small,left_on=[7],right_on=['Id'],how='left',suffixes=('_4','_5'))
df5.sort_values(by=['val'],ascending=False).to_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\route_base_short.csv',index=False)

In [232]:
df_re_new.loc[df_re_new[0]=='C3130']

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,val
118536,C3130,I6531,0.052632,I6531,C3940,0.800000,C1371,C2431,0.062500,C2431,C3940,0.200000,0.0,0.0,0.000000,0.0,0.0,0.0,1.115132
118537,C3130,I6531,0.052632,I6531,C3940,0.800000,C1371,C2023,1.454545,C2023,C3940,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,2.557177
118538,C3130,C2624,0.727273,C2624,C3940,0.125000,C1371,C2431,0.062500,C2431,C3940,0.200000,0.0,0.0,0.000000,0.0,0.0,0.0,1.114773
118539,C3130,C2624,0.727273,C2624,C3940,0.125000,C1371,C2023,1.454545,C2023,C3940,0.250000,0.0,0.0,0.000000,0.0,0.0,0.0,2.556818
118540,C3130,K7010,3.600000,K7010,C3940,0.166667,C1371,C2431,0.062500,C2431,C3940,0.200000,0.0,0.0,0.000000,0.0,0.0,0.0,4.029167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122863,C3130,C2311,0.181818,C2311,C3253,0.166667,C3253,C1314,1.000000,C3972,C1314,0.500000,0.0,0.0,0.000000,0.0,0.0,0.0,1.848485
122864,C3130,C2311,0.181818,C2311,C3253,0.166667,C3253,C1314,1.000000,L7292,C1314,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,1.681818
122865,C3130,C2311,0.181818,C2311,C3253,0.166667,C3253,C1314,1.000000,C1314,L7283,1.000000,C1371,L7283,0.333333,0.0,0.0,0.0,2.681818
122866,C3130,C2311,0.181818,C2311,C3253,0.166667,C3253,C1314,1.000000,C1314,L7283,1.000000,C1523,L7283,1.000000,0.0,0.0,0.0,3.348485


In [233]:
len(np.where(df_re_new=='C3130')[0])

2112

In [238]:
df_re_short=df_re.loc[df_re[9]==0]
df_re_short.sort_values(by=['val'],ascending=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,val
49236,C2013,C2661,6.000000,L7292,M7484,4.000000,M7484,C2661,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000
67527,C2013,C2661,6.000000,L7292,M7484,4.000000,M7484,C2661,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000
50794,C2013,E4910,10.562500,C3660,E4910,1.388889,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.951389
134342,C3252,C3110,6.000000,C1771,I6513,3.600000,I6513,C3110,2.285714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.885714
145059,C3252,C3110,6.000000,C1771,I6513,3.600000,I6513,C3110,2.285714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.885714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92237,C2740,C2914,0.050000,C3660,C2914,0.058824,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108824
79060,C2223,C3515,0.058824,C3824,C3515,0.050000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108824
70496,C2221,E4920,0.055556,C1771,E4920,0.052632,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108187
96440,C2740,C1392,0.038462,C2110,C1392,0.062500,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100962


In [69]:
# 联系数据重整，筛选五步及以下的联系，按照起点，起点名称，强度，终点，终点名称的形式进行整理
df_re=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\route_base.csv')
df_re1=df_re.drop_duplicates()
df_re_short=df_re1.loc[df_re1['15']=='0.0']
ls_re_short=df_re_short.values.tolist()

In [72]:
df_re_short

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,val
0,A0133,A0123,0.066667,C1371,A0123,0.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.816667
1,A0133,A0123,0.066667,C1421,A0123,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.466667
2,A0133,A0123,0.066667,C3841,A0123,0.400000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.466667
3,A0133,A0123,0.066667,C3973,A0123,0.900000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.966667
4,A0133,A0123,0.066667,L7292,A0123,0.266667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143968,C3252,C3032,1.125000,C3032,C3253,0.125000,C3253,C1314,1.0,C1314,L7283,1.0,C3841,L7283,1.0,0.0,0.0,0.0,4.250000
143969,C3252,C1783,1.333333,C1783,C3253,0.571429,C3253,C1314,1.0,C1314,L7283,1.0,C3841,L7283,1.0,0.0,0.0,0.0,4.904762
143970,C3252,C3360,0.200000,C3360,C3253,1.285714,C3253,C1314,1.0,C1314,L7283,1.0,C3841,L7283,1.0,0.0,0.0,0.0,4.485714
143971,C3252,C3351,1.285714,C3351,C3253,0.111111,C3253,C1314,1.0,C1314,L7283,1.0,C3841,L7283,1.0,0.0,0.0,0.0,4.396825


In [78]:
ls_re_short=df_re_short.values.tolist()
ls_short=[]
for i in ls_re_short:
    ls_i=[]
    i_1=[x for x in i if type(x)==float and x!=0][:-1]
    i_2=[x for x in i if type(x)==str and x!='0.0']
    ls_pair=[i_2[x:x+2] for x in range(0,len(i_2),2)]
    
    # print(i_1)
    # print(i_2)
    print(len(ls_pair))
    
    for a in range(len(ls_pair)-1):
        ls_a=ls_pair[a]
        ls_a_1=ls_pair[a+1]
        ls_a_2=ls_pair[-1]
        if ls_a[-1]==ls_a_1[0]:
            continue
        elif ls_a[-1]==ls_a_2[-1]:
            break
    ls_pair_re1=ls_pair[:a+1]
    ls_pair_re2=ls_pair[a+1:]
    ls_pair_re3=[list(reversed(x)) for x in list(reversed(ls_pair_re2))]
    ls_pair_re1.extend(ls_pair_re3)
    ls_pair_re4=[y for x in ls_pair_re1 for y in x]
    first_dict={}
    ls_pair_re5=list(first_dict.fromkeys(ls_pair_re4).keys())
    
    ls_i.append(ls_pair_re5[0])
    for b in range(1,5):
        if b < len(ls_pair_re5)-1:
            ls_i.append(i_1[b-1])
            ls_i.append(ls_pair_re5[b])
        else:
            ls_i.append(0)
            ls_i.append(0)
    ls_i.append(i_1[-1])
    ls_i.append(ls_pair_re5[-1])
    ls_short.append(ls_i)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [86]:
df_re_short_new=pd.DataFrame(ls_short,columns=['ind_o','weight_1','ind_1','weight_2','ind_2','weight_3','ind_3','weight_4','ind_4','weight_d','ind_d'])
df0=pd.merge(left=df_re_short_new,right=df_small,left_on=['ind_o'],right_on=['Id'],how='left',suffixes=('_0','_1'))
df1=pd.merge(left=df0,right=df_small,left_on=['ind_1'],right_on=['Id'],how='left',suffixes=('_0','_1'))
df2=pd.merge(left=df1,right=df_small,left_on=['ind_2'],right_on=['Id'],how='left',suffixes=('_0','_1'))
df3=pd.merge(left=df2,right=df_small,left_on=['ind_3'],right_on=['Id'],how='left',suffixes=('_2','_3'))
df4=pd.merge(left=df3,right=df_small,left_on=['ind_4'],right_on=['Id'],how='left',suffixes=('_4','_5'))
df5=pd.merge(left=df4,right=df_small,left_on=['ind_d'],right_on=['Id'],how='left',suffixes=('_4','_5'))
df6=df5.loc[:,['ind_o','Label_0','weight_1','ind_1','Label_1','weight_2','ind_2','Label_2','weight_3','ind_3','Label_3','weight_4','ind_4','Label_4','weight_d','ind_d','Label_5']]
df6.loc[:,'val']=df_re_short['val']
df6.sort_values(by=['val'],ascending=False).to_csv(r'E:\project_basedata\2023090801产业生长图谱\ind_example\route_base_short_new.csv',index=False)

In [88]:
ls1=df6['ind_o'].unique().tolist()
[x for x in ls1 if x not in ls_adv_1]

[]

In [ ]:
df_result=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\small_rca_revolve_1990_2020_poly.csv',dtype={'area_city':'str'})
ls_se=['A','B','C','E','G','I','J','K','L','M','R']
df_result.loc[:,'ind_type']=df_result['ind'].str[0]
df_result_new=df_result.loc[df_result['ind_type'].isin(ls_se)]

df_new_result_2_filt_new1=df_new_result_2_filt_new.sort_values(['primary_weight'],ascending=False).reset_index(drop=True)
for i in range(10):
    city=df_new_result_2_filt_new1.loc[i,'city_name']
    ls_city=city.split(';')
    slope_o=df_new_result_2_filt_new1.loc[i,'slope_o']
    ls_slope_o=slope_o.split(';')
    slope_d=df_new_result_2_filt_new1.loc[i,'slope_d']
    ls_slope_d=slope_d.split(';')
    
    for a in range(len(ls_slope_d)):
        if ls_slope_o[a]!='no' and ls_slope_d[a]!='no' and float(ls_slope_o[a])<0 and float(ls_slope_d[a])>0:
            city_name=ls_city[a]
            ind_a=df_new_result_2_filt_new1.loc[i,'primary_code']
            ind_b=df_new_result_2_filt_new1.loc[i,'related_code']
            ind_a_name=df_new_result_2_filt_new1.loc[i,'real_o']
            ind_b_name=df_new_result_2_filt_new1.loc[i,'real_d']
            df_i=df_result_new.loc[df_result_new['admin_name']==city_name]
            df_ab=df_i.loc[(df_i['ind']==ind_a)|(df_i['ind']==ind_b)]
            df_2=df_ab.dropna(axis=1,how='any')
            ls_col=df_2.columns.tolist()
            ls_col_se=[x for x in ls_col if x not in ['area_city','ind','admin_name','Label','ind_type','line_coef_1','line_coef_2','r_square']]
            df_3=df_2.set_index(['ind'])
            df_3_data=df_3[ls_col_se].transpose()
            if len(df_3_data)<=6:
                continue
            else:
                xdata=df_3_data.index.tolist()
                ydata1=df_3_data[ind_a].tolist()
                ydata2=df_3_data[ind_b].tolist()
                
                fig = plt.figure(figsize=(8, 6), dpi=100)
                ax=fig.add_subplot()
                # ax.axis('equal')
                ax.plot(xdata,ydata1,label=ind_a)
                ax.plot(xdata,ydata2,label=ind_b)
                ax.legend()
                plt.savefig('E:\\project_basedata\\2023090801产业生长图谱\\ind_example\\'+city_name+'_'+ind_a+'_'+ind_a_name+'_'+ind_b+ind_b_name+'.png')
                plt.close()

In [2]:
df_new_result_2=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\负相关联系方向编码及平方频数筛选基底.csv')
df_result=pd.read_csv(r'E:\project_basedata\2023090801产业生长图谱\small_rca_revolve_1990_2020_poly.csv',dtype={'area_city':'str'})
ls_graph_ind=[['A0133','C3561'],['B1011','J6712'],['I6520','C1771'],['C3611','C1523'],['A0171','L7292'],['C3011','I6560'],['C3021','I6312'],['I6560','L7292']]
for i in ls_graph_ind:
    ind_o=i[0]
    ind_d=i[1]
    df_i=df_new_result_2.loc[(df_new_result_2['real_o_code']==ind_o)&(df_new_result_2['real_d_code']==ind_d)]
    if len(df_i)==1:
        city=df_i['city_name'].values.tolist()[0]
        ls_city=city.split(';')
        slope_o=df_i['slope_o'].values.tolist()[0]
        ls_slope_o=slope_o.split(';')
        slope_d=df_i['slope_d'].values.tolist()[0]
        ls_slope_d=slope_d.split(';')
        print(ls_city)
        print(ls_slope_o)
        print(ls_slope_d)
        for a in range(len(ls_slope_d)):
            if ls_slope_o[a]!='no' and ls_slope_d[a]!='no':
                city_name=ls_city[a]
                if float(ls_slope_o[a])<0 and float(ls_slope_d[a])>0:
                    ind_a=df_i['real_o_code'].values.tolist()[0]
                    ind_b=df_i['real_d_code'].values.tolist()[0]
                    ind_a_name=df_i['real_o'].values.tolist()[0]
                    ind_b_name=df_i['real_d'].values.tolist()[0]
                    
                    
                elif float(ls_slope_o[a])>0 and float(ls_slope_d[a])<0:
                    ind_a=df_i['real_d_code'].values.tolist()[0]
                    ind_b=df_i['real_o_code'].values.tolist()[0]
                    ind_a_name=df_i['real_d'].values.tolist()[0]
                    ind_b_name=df_i['real_o'].values.tolist()[0]
                    
                df_iab=df_result.loc[df_result['admin_name']==city_name]
                df_ab=df_iab.loc[(df_iab['ind']==ind_a)|(df_iab['ind']==ind_b)]

                df_2=df_ab.dropna(axis=1,how='any')
                ls_col=df_2.columns.tolist()
                ls_col_se=[x for x in ls_col if x not in ['area_city','ind','admin_name','Label','ind_type','line_coef_1','line_coef_2','r_square']]
                df_3=df_2.set_index(['ind'])
                df_3_data=df_3[ls_col_se].transpose()
                if len(df_3_data)<=6:
                    continue
                else:
                    xdata=df_3_data.index.tolist()
                    ydata1=df_3_data[ind_a].tolist()
                    ydata2=df_3_data[ind_b].tolist()

                    fig = plt.figure(figsize=(8, 6), dpi=200)
                    ax=fig.add_subplot()
                    # ax.axis('equal')
                    ax.plot(xdata,ydata1,label=ind_a)
                    ax.plot(xdata,ydata2,label=ind_b)
                    ax.legend()
                    plt.savefig('E:\\project_basedata\\2023090801产业生长图谱\\ind_example\\'+city_name+'_'+ind_a+'_'+ind_a_name+'_'+ind_b+'_'+ind_b_name+'.png')
                    plt.close()
    else:
        print('error1')

['宁波市', '广州市', '惠州市']
['-0.4481', '-0.3339', 'no']
['no', '0.0583', 'no']
['娄底市', '抚州市', '晋城市', '济宁市', '淮南市']
['1.0338', 'no', 'no', 'no', '-0.1624']
['no', 'no', '0.2567', 'no', 'no']
['大同市', '平顶山市', '徐州市', '德州市', '日照市', '昭通市', '泰安市', '淄博市', '烟台市', '衡阳市', '邯郸市', '长春市']
['0.1359', 'no', 'no', '0.2995', 'no', '0.4805', 'no', '0.2071', 'no', 'no', '0.2331', '0.0553']
['no', '-0.21', 'no', 'no', '-0.5576', 'no', 'no', 'no', 'no', '-0.2286', '-0.0984', 'no']
['淄博市', '潍坊市', '聊城市']
['no', 'no', 'no']
['no', 'no', '-1.5442']
['太原市', '宿州市', '来宾市', '楚雄彝族自治州']
['-0.1012', 'no', '0.2412', '0.627']
['no', '0.1117', 'no', 'no']
['四平市', '巴音郭楞蒙古自治州', '德宏傣族景颇族自治州', '淄博市', '湖州市', '白城市', '许昌市', '赤峰市']
['no', 'no', 'no', '-0.0785', 'no', 'no', 'no', 'no']
['no', 'no', 'no', '0.045', 'no', 'no', '0.0559', 'no']
['云浮市', '厦门市', '青岛市']
['no', '-0.0174', 'no']
['no', '0.0109', '0.0177']
['呼伦贝尔市', '大同市', '忻州市', '晋中市', '朔州市', '泸州市', '运城市']
['no', '-0.3798', '-0.2774', 'no', 'no', 'no', 'no']
['no', 'no', 'no', 

In [104]:
df_new_result_2.loc[(df_new_result_2['real_o_code']=='I6520')&(df_new_result_2['real_d_code']=='C1771')]

,ind_o_,ind_d_,admin_name,rate,city_name,line_coef_1_,ind_name_o,ind_name_d,area_city,area_city_name,slope_o,slope_d,real_o,real_d,rate_new,real_o_code,real_d_code,ind_o_type,ind_d_type,weight
44532,C1771,I6520,12,0.032967,大同市;平顶山市;徐州市;德州市;日照市;昭通市;泰安市;淄博市;烟台市;衡阳市;邯郸市;长春市,-2.702294,床上用品制造,集成电路设计,14,呼和浩特市;大同市;平顶山市;徐州市;德州市;日照市;昭通市;泰安市;淄博市;烟台市;衡阳市...,0.1359;no;no;0.2995;no;0.4805;no;0.2071;no;no;...,no;-0.21;no;no;-0.5576;no;no;no;no;-0.2286;-0....,集成电路设计,床上用品制造,0.857143,I6520,C1771,C,I,10.285714


In [94]:
df_new_result_2.loc[(df_new_result_2['real_o_code']=='A0133')&(df_new_result_2['real_d_code']=='C3561')]['city_name'].values.tolist()[0]

['宁波市;广州市;惠州市']

In [91]:
df_new_result_2

,ind_o_,ind_d_,admin_name,rate,city_name,line_coef_1_,ind_name_o,ind_name_d,area_city,area_city_name,slope_o,slope_d,real_o,real_d,rate_new,real_o_code,real_d_code,ind_o_type,ind_d_type,weight
0,A0111,A0112,3,0.008242,仙桃市;淮南市;铜陵市,-0.859162,稻谷种植,小麦种植,25,临汾市;临沂市;仙桃市;保定市;北京市;台州市;吉林市;喀什地区;安庆市;宜昌市;德州市;恩...,no;0.3868;0.3078,no;no;no,小麦种植,稻谷种植,0.120000,A0112,A0111,A,A,0.360000
1,A0111,A0113,4,0.010989,东方市;信阳市;周口市;银川市,-1.038343,稻谷种植,玉米种植,33,三门峡市;东方市;保定市;信阳市;博尔塔拉蒙古自治州;吉林市;周口市;四平市;宜昌市;庆阳市...,no;no;no;no,no;no;no;0.0511,稻谷种植,玉米种植,0.121212,A0111,A0113,A,A,0.484848
2,A0111,A0121,3,0.008242,张家界市;泉州市;濮阳市,-0.641871,稻谷种植,豆类种植,27,伊春市;凉山彝族自治州;南充市;南通市;安顺市;宜昌市;宿州市;广元市;延边朝鲜族自治州;张...,0.4202;no;no,-0.069;0.0361;no,豆类种植,稻谷种植,0.111111,A0121,A0111,A,A,0.333333
3,A0111,A0122,3,0.008242,中卫市;佳木斯市;绥化市,-0.916370,稻谷种植,油料种植,25,中卫市;佳木斯市;南充市;南通市;吉林市;宿迁市;巴中市;广元市;新余市;株洲市;梅州市;永...,no;1.0012;1.2553,no;-2.3236;no,油料种植,稻谷种植,0.120000,A0122,A0111,A,A,0.360000
4,A0111,A0123,5,0.013736,丹东市;信阳市;厦门市;聊城市;芜湖市,-1.927193,稻谷种植,薯类种植,29,三门峡市;丹东市;乌兰察布市;信阳市;南通市;厦门市;双河市;喀什地区;宁波市;安顺市;宿州...,0.0974;no;no;0.0501;no,no;no;no;no;no,薯类种植,稻谷种植,0.172414,A0123,A0111,A,A,0.862069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137008,M7516,M7520,2,0.005495,阜新市;黄石市,-0.136922,环保技术推广服务,知识产权服务,7,桂林市;永州市;漯河市;焦作市;阜新市;黄石市;齐齐哈尔市,no;-0.7193,no;-0.0678,知识产权服务,环保技术推广服务,0.285714,M7520,M7516,M,M,0.571429
137009,M7516,M7540,1,0.002747,晋中市,-1.093158,环保技术推广服务,创业空间服务,1,晋中市,-0.5958,-3.7073,创业空间服务,环保技术推广服务,1.000000,M7540,M7516,M,M,1.000000
137010,M7517,M7540,1,0.002747,郑州市,-0.189909,三维(3D)打印技术推广服务,创业空间服务,1,郑州市,-8.1356,no,三维(3D)打印技术推广服务,创业空间服务,1.000000,M7517,M7540,M,M,1.000000
137011,M7520,M7530,3,0.008242,景德镇市;赤峰市;鹰潭市,-1.283247,知识产权服务,科技中介服务,23,东莞市;东营市;伊春市;佛山市;六盘水市;合肥市;孝感市;景德镇市;泸州市;淮南市;红河哈尼...,no;no;no,-0.122;no;-0.0976,科技中介服务,知识产权服务,0.130435,M7530,M7520,M,M,0.391304
